[View in Colaboratory](https://colab.research.google.com/github/JacksonIsaac/colab_notebooks/blob/master/Chap2_machine_learning_in_action.ipynb)

# Machine Learning in Action

## Chapter 2

### k-Nearest Neighbours

We cluster the data based on the closeness of data points to each other using distance measurement. In simple words, grouping data points together which have minimal distance between them.

When a new data input is given, we calculate its distance from each cluster points, and select the most similar clusters (nearest neighbours). From this, we look at the top *k* clusters.

**Pros:** High accuracy, insensitive to outliers, no assumptions about data

**Cons:** Computationally expensive, requires a lot of memory

**Works with:** Numeric values, nominal values

## Import Dependencies

In [0]:
import numpy as np
import operator

### Let's create a sample dataset

In [0]:
def create_dataset():
    group = np.array([[1., 1.1], [1., 1.], [0, 0], [0, 0.1]])
    labels = ['A', 'A', 'B', 'B']
    return group, labels

In [0]:
group, labels = create_dataset()

In [12]:
group

array([[1. , 1.1],
       [1. , 1. ],
       [0. , 0. ],
       [0. , 0.1]])

In [13]:
labels

['A', 'A', 'B', 'B']

## Classification using kNN

For every datapoint we will:


*   Calculate the distance between input data point (inX) and each point in trained dataset matrix with corresponding classification labels
*   Sort distances in increasing order
*   Select *k* items with lowest distances to inX
*   Find majority of items among these *k* items
*   Return the majority class as the classification of inX.



### Distance calculation logic

Distance between data points is calculated by using Euclidian distance.

![Euclidian distance formula](https://i.imgur.com/I9sKgfn.png)